In [1]:
import requests
import psycopg2
import pandas as pd
from passwords_weather import api_key
import io


In [60]:
# Функция для определения типов данных
def infer_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return "INTEGER"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "TIMESTAMP"
    else:
        return "TEXT"

In [61]:
#Создание таблицы
def create_table(df, table_name):
    # Подключение к базе данных PostgreSQL
    conn = psycopg2.connect(
        dbname="database_weather",
        user="dvorak1981",
        password="admin",
        host="172.19.0.3",
        port="5432",
        connect_timeout=10  # Устанавливаем таймаут
    )
    cursor=conn.cursor()
    
    # Генерируем SQL-запрос для создания таблицы
    columns_sql=", ".join([f'"{col}" {infer_sql_type(df[col])}' for col in df.columns])
    create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql});"
    #Создаём таблицу
    cursor.execute(create_table_sql)
    conn.commit()
    
    cursor.close()
    conn.close()
    print(f'Таблица {table_name} создана или уже существует')

In [62]:
# Загружаем данные в PostgreSQL
def load_data_in_postgerSQL (df, table_name):
        # Подключение к базе данных PostgreSQL
    conn = psycopg2.connect(
        dbname="database_weather",
        user="dvorak1981",
        password="admin",
        host="172.19.0.3",
        port="5432",
        connect_timeout=10  # Устанавливаем таймаут
    )
    cursor=conn.cursor()
    
    for _, row in df.iterrows():
        values = ', '.join(["%s"] * len(row)) #%s Предотвращает SQL-инъекции
        insert_sql= f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({values})"
        cursor.execute(insert_sql, tuple(row))
        conn.commit()
    
    cursor.close()
    conn.close()

In [63]:
def get_weather_in_city_csv(city = "Moscow"):
    params = {
        "unitGroup": "metric",
        "key": api_key,
        "contentType": "csv",
        "include": "days"
    }

    URL_base = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/last30days"
    response = requests.get(URL_base, params=params)


    # Выводим итоговый URL
    print(f"Запрос: {response.request.url}")

    if response.status_code == 200:
        name_of_table = f"{city}_weather_for_last_30_days"
        df=pd.read_csv(io.BytesIO(response.content))
        create_table(df, name_of_table)
        load_data_in_postgerSQL(df, name_of_table)
        print(f"Данные {name_of_table} успешно сохранёны в PostgreSQL ")
    else:
        print(f"Ошибка запроса для {city}: {response.status_code}, {response.text}")

In [64]:
cities= ['Moscow', 'London', 'Kerch', 'Krasnodar']
list(map(get_weather_in_city_csv, cities))

Запрос: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Moscow/last30days?unitGroup=metric&key=ZJDQ2FLBSC67M32HYFY6VP3VK&contentType=csv&include=days
Таблица Moscow_weather_for_last_30_days создана или уже существует
Данные Moscow_weather_for_last_30_days успешно сохранёны в PostgreSQL 
Запрос: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/London/last30days?unitGroup=metric&key=ZJDQ2FLBSC67M32HYFY6VP3VK&contentType=csv&include=days
Таблица London_weather_for_last_30_days создана или уже существует
Данные London_weather_for_last_30_days успешно сохранёны в PostgreSQL 
Запрос: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Kerch/last30days?unitGroup=metric&key=ZJDQ2FLBSC67M32HYFY6VP3VK&contentType=csv&include=days
Таблица Kerch_weather_for_last_30_days создана или уже существует
Данные Kerch_weather_for_last_30_days успешно сохранёны в PostgreSQL 
Запрос: https://weather.visualc

[None, None, None, None]

In [4]:
# Сохранение таблиц в csv файлы на локальную машину
conn=psycopg2.connect(
    dbname='database_weather',
    user='dvorak1981',
    password='admin',
    host='172.19.0.3',
    port='5432'
)

# SQL-запрос для получения списка таблиц

query="""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
"""

# Читаем данные в pandas DataFrame

df_tables = pd.read_sql(query, conn)

table_names=df_tables["table_name"].tolist()
print(table_names)

for table_name in table_names:
    df=pd.read_sql(f"SELECT * FROM {table_name};", conn)
    # Сохраняем в CSV
    df.to_csv(f"{table_name}.csv", index = False)

conn.close()

['kerch_weather_for_last_30_days', 'krasnodar_weather_for_last_30_days', 'london_weather_for_last_30_days', 'moscow_weather_for_last_30_days']


/tmp/ipykernel_169/354708582.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tables = pd.read_sql(query, conn)
/tmp/ipykernel_169/354708582.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(f"SELECT * FROM {table_name};", conn)
